In [1]:
import os

# 환경 변수 설정
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

In [2]:
import pandas as pd
import numpy as np
import random

import gymnasium as gym
from gymnasium import spaces
import pybamm

from stable_baselines3 import DDPG, PPO
from stable_baselines3.common.env_checker import check_env
from stable_baselines3.common.noise import NormalActionNoise, OrnsteinUhlenbeckActionNoise

In [3]:
params = pybamm.ParameterValues("Chen2020").copy()

In [4]:
params

{'Ambient temperature [K]': 298.15,
 'Boltzmann constant [J.K-1]': 1.380649e-23,
 'Bulk solvent concentration [mol.m-3]': 2636.0,
 'Cation transference number': 0.2594,
 'Cell cooling surface area [m2]': 0.00531,
 'Cell thermal expansion coefficient [m.K-1]': 1.1e-06,
 'Cell volume [m3]': 2.42e-05,
 'Contact resistance [Ohm]': 0,
 'Current function [A]': 5.0,
 'EC diffusivity [m2.s-1]': 2e-18,
 'EC initial concentration in electrolyte [mol.m-3]': 4541.0,
 'Electrode height [m]': 0.065,
 'Electrode width [m]': 1.58,
 'Electrolyte conductivity [S.m-1]': <function electrolyte_conductivity_Nyman2008 at 0x000002E9D6701F30>,
 'Electrolyte diffusivity [m2.s-1]': <function electrolyte_diffusivity_Nyman2008 at 0x000002E9D6701EA0>,
 'Electron charge [C]': 1.602176634e-19,
 'Faraday constant [C.mol-1]': 96485.33212,
 'Ideal gas constant [J.K-1.mol-1]': 8.314462618,
 'Initial concentration in electrolyte [mol.m-3]': 1000.0,
 'Initial concentration in negative electrode [mol.m-3]': 29866.0,
 'Initi

In [7]:
class DFN(gym.Env):
    def __init__(self, render_mode=None):
        options = {"thermal": "lumped"}
        self.model = pybamm.lithium_ion.SPMe(options)

        self.params = pybamm.ParameterValues("Chen2020").copy()
        init_input = {
            'Number of cells connected in series to make a battery': 96,
            'Upper voltage cut-off [V]': 5,
        }
        self.params.update(init_input)
        
        geometry = self.model.default_geometry 
        submesh_types = self.model.default_submesh_types 
        var_pts = self.model.default_var_pts 
        spatial_methods = self.model.default_spatial_methods 
        
        self.params.process_geometry(geometry)
        mesh = pybamm.Mesh(geometry, submesh_types, var_pts) 
        self.disc = pybamm.Discretisation(mesh, spatial_methods)

        self.solutions = []

        self.r_max_temp = 273 + 35
        self.r_max_volt = 4.2
        self.SoC_desired = 0.8

        self.volt = 0
        self.temp = 0
        self.SoC = 0
        
        self.time_goal = 0
        self.ep_num = 0
        self.time_step = 0
        self.MAX_time_step = 3600*2

        
        self.observation_space = spaces.Box(low=0, high=400, shape=(5,), dtype=np.float32)
        self.action_space = spaces.Box(dtype=np.float32, low=-800, high=-1, shape=(1,))
        
    def make_new_model(self, update_input):
        model1 = self.model.new_copy()
        param1 = self.params.copy()
        
        param1.update(update_input)
        model1 = param1.process_model(model1, inplace=False) # 파라미터에 모델 변화
        built_model = self.disc.process_model(model1, inplace=True, check_model=True) # 모델을 프로세싱
        return built_model
    
    def update_model_step(self,input):
        model = self.make_new_model(input)
        solver = pybamm.CasadiSolver(mode="fast")
        if len(self.solutions) == 0:
            self.SoC = 0.2
            experiment = pybamm.Experiment(["Rest for 30 min"])
            sim = pybamm.Simulation(self.model, experiment=experiment)
            step_solution = sim.solve(initial_soc=0.2)
        else:
            
            step_solution = solver.step(self.solutions[-1].last_state,
                                        model,
                                        30,
                                        npts=30,
                                        save=False,)
        self.solutions += [step_solution]
        return step_solution

    def step(self, action):
            
        new_input = {
            "Current function [A]": float(action)
        }
        try:
            solution = self.update_model_step(new_input)
            
            self.temp = solution["X-averaged cell temperature [K]"].entries[-1]
            self.volt = solution["Terminal voltage [V]"].entries[-1]     
            Q = self.params["Nominal cell capacity [A.h]"]
            DC = solution["Discharge capacity [A.h]"].entries[-1]
            self.SoC = self.SoC-DC/Q

            if self.SoC >= self.SoC_desired:
                terminated = True

            else:
                terminated = False
            # Calculate reward based on various factors
            if self.time_step >= self.MAX_time_step:
                terminated = True
                reward = -10000
            
            r_temp = -5 * abs(self.temp - (273+35)) if self.temp> (273+35) else 0
            r_volt = -200 * abs(self.volt - self.r_max_volt) if self.volt > self.r_max_volt else 0

            r_step = -0.01 if self.time_step >= self.time_goal else -0.1*abs(self.time_goal - self.time_step)
            r_soc =  -10 * abs(self.SoC - self.SoC_desired + 1) if self.time_step  > self.time_goal  else 0 
            
            reward = r_step +r_temp +r_volt + r_soc 

            reward = float(reward)               
            # Check if termination condition is met
            
            self.time_step +=1
        except:
            terminated = True
            reward = -10000

        observation = self._get_obs()
        info = self._get_info()
        print(self.time_step , observation,reward,"|",float(action))
        return observation, reward, terminated, False, info

    def reset(self, seed=None, options=None):
        print(self.time_step)
        self.time_goal = self.generate_random_number()
        super().reset(seed=seed)
        print("reset==================================")
        print(self.time_goal)
        self.solutions = []
        self.SoC = 0.2
        experiment = pybamm.Experiment(["Rest for 30 min"])
        sim = pybamm.Simulation(self.model, experiment=experiment)
        step_solution = sim.solve(initial_soc=0.2)
        self.solutions += [step_solution]
        observation = self._get_obs()
        info = self._get_info()
        self.ep_num +=1
        self.time_step = 0
        return observation, info
    
    def generate_random_number(self):
        return int(random.random()*100+20)*2 

    def _get_obs(self):
        return np.array([self.SoC,self.volt,self.temp,self.time_goal,self.time_step], dtype=np.float32)

    def _get_info(self):
        return {"distance": self.SoC_desired - self.SoC}

env = DFN()
# It will check your custom environment and output additional warnings if needed
check_env(env)  

0
reset==================================
230
0
reset==================================
100
1 [2.2064088e-01 5.0201235e+00 3.0374146e+02 1.0000000e+02 1.0000000e+00] -174.0246639784662
1
reset==================================
128
0 [2.0000000e-01 5.0201235e+00 3.0374146e+02 1.2800000e+02 0.0000000e+00] -10000
0
reset==================================
82
0 [2.0000000e-01 5.0201235e+00 3.0374146e+02 8.2000000e+01 0.0000000e+00] -10000
0
reset==================================
112
0 [2.0000000e-01 5.0201235e+00 3.0374146e+02 1.1200000e+02 0.0000000e+00] -10000
0
reset==================================
160
0 [2.0000000e-01 5.0201235e+00 3.0374146e+02 1.6000000e+02 0.0000000e+00] -10000
0
reset==================================
68
1 [2.1942900e-01 4.9571605e+00 3.0477823e+02 6.8000000e+01 1.0000000e+00] -158.23209282277912
2 [2.1942900e-01 4.9571605e+00 3.0477823e+02 6.8000000e+01 2.0000000e+00] -158.1320928227791
3 [2.1942900e-01 4.9571605e+00 3.0477823e+02 6.8000000e+01 3.0000000e+00] -1

In [8]:
# The noise objects for DDPG


model = PPO("MlpPolicy", env,  verbose=1)
model.learn(total_timesteps=300000, log_interval=10)
model.save("ddpg_pendulum")
vec_env = model.get_env()


Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
6
reset==================================
72
1 [2.0166667e-01 3.5701871e+00 2.9816916e+02 7.2000000e+01 1.0000000e+00] -7.2
2 [2.0333333e-01 3.5774555e+00 2.9819199e+02 7.2000000e+01 2.0000000e+00] -7.1000000000000005
3 [2.0500000e-01 3.5819323e+00 2.9821576e+02 7.2000000e+01 3.0000000e+00] -7.0
4 [2.0666666e-01 3.5851483e+00 2.9823941e+02 7.2000000e+01 4.0000000e+00] -6.9
5 [2.1065244e-01 3.6415083e+00 2.9836761e+02 7.2000000e+01 5.0000000e+00] -6.800000000000001
6 [2.1231911e-01 3.5974462e+00 2.9838882e+02 7.2000000e+01 6.0000000e+00] -6.7
7 [2.1398577e-01 3.5972211e+00 2.9840765e+02 7.2000000e+01 7.0000000e+00] -6.6000000000000005
8 [2.1565244e-01 3.5982409e+00 2.9842508e+02 7.2000000e+01 8.0000000e+00] -6.5
9 [2.1731910e-01 3.5997415e+00 2.9844159e+02 7.2000000e+01 9.0000000e+00] -6.4
10 [2.1898577e-01 3.6014426e+00 2.9845731e+02 7.2000000e+01 1.0000000e+01] -6.300000000000001
11 [2.206524